# Geospatial Project 

In [136]:
#First step is to import the libraries I will use
import pandas as pd
import os
import geopandas as gpd
from pymongo import MongoClient
import requests
import json 
from dotenv import load_dotenv
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from functools import reduce
import operator
import cartoframes
from cartoframes.viz import Map, Layer, popup_element


# Office Location

I use MongoDB to look for companies, mainly their categories and their location

In [2]:
client = MongoClient("mongodb://localhost/ironhack")
db = client.get_database()

In [3]:
list(db.list_collections())

[{'name': 'companies',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('f0df70fa-ecbb-4210-b11a-d2f3d3075dc8')},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}]

In [4]:
db.companies.distinct("category_code") 

[None,
 'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'government',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web']

I want to be near web companies so I will make a dataframe with all of the web companies. <br>
I also choose London as the city where I want my office to be because it is one of the central hubs of Europe,

In [138]:
#I get the web companies in London
query1 = {"category_code":"web", "offices.city": {"$eq":"London"}}
project = {"_id":0, "offices.city":1, "offices.latitude":1, "offices.longitude":1}
res = db.companies.find(query1,project)

In [6]:
res

In [7]:
df_loc = list(res)
df_loc = pd.DataFrame.from_dict(df_loc)

In [139]:
#Dataframe with the offices that have a location in London
df_loc


,offices
0,"[{'city': 'San Jose', 'latitude': 37.321798, '..."
1,"[{'city': 'Los Angeles', 'latitude': 37.09024,..."
2,"[{'city': 'London', 'latitude': 37.09024, 'lon..."
3,"[{'city': 'London', 'latitude': 51.5192042, 'l..."
4,"[{'city': 'London', 'latitude': 37.09024, 'lon..."
...,...
107,"[{'city': 'London', 'latitude': 51.5215279, 'l..."
108,"[{'city': 'London', 'latitude': None, 'longitu..."
109,"[{'city': 'London', 'latitude': None, 'longitu..."
110,"[{'city': '', 'latitude': None, 'longitude': N..."


In [137]:
#I get all the web companies
query = {"category_code":"web"}
project = {"_id":0, "name":1, "category_code":1}

res = db.companies.find(query,project)
res = list(res)
res[:5]

[{'name': 'Wetpaint', 'category_code': 'web'},
 {'name': 'Postini', 'category_code': 'web'},
 {'name': 'Geni', 'category_code': 'web'},
 {'name': 'Fox Interactive Media', 'category_code': 'web'},
 {'name': 'StumbleUpon', 'category_code': 'web'}]

In [10]:
df_res = pd.DataFrame(res)

In [11]:
df_res

,name,category_code
0,Wetpaint,web
1,Postini,web
2,Geni,web
3,Fox Interactive Media,web
4,StumbleUpon,web
...,...,...
3782,Optemo,web
3783,Zoomission,web
3784,Eazeeloans,web
3785,DocASAP,web


In [140]:
#I put together the dataframes of the location and the name and category of the web companies
result = pd.concat([df_res, df_loc], axis=1)

In [13]:
result

,name,category_code,offices
0,Wetpaint,web,"[{'city': 'San Jose', 'latitude': 37.321798, '..."
1,Postini,web,"[{'city': 'Los Angeles', 'latitude': 37.09024,..."
2,Geni,web,"[{'city': 'London', 'latitude': 37.09024, 'lon..."
3,Fox Interactive Media,web,"[{'city': 'London', 'latitude': 51.5192042, 'l..."
4,StumbleUpon,web,"[{'city': 'London', 'latitude': 37.09024, 'lon..."
...,...,...,...
3782,Optemo,web,NaN
3783,Zoomission,web,NaN
3784,Eazeeloans,web,NaN
3785,DocASAP,web,NaN


In [14]:
result1 = result.explode('offices')
result1

,name,category_code,offices
0,Wetpaint,web,"{'city': 'San Jose', 'latitude': 37.321798, 'l..."
0,Wetpaint,web,"{'city': 'London', 'latitude': 51.500152, 'lon..."
1,Postini,web,"{'city': 'Los Angeles', 'latitude': 37.09024, ..."
1,Postini,web,"{'city': 'London', 'latitude': None, 'longitud..."
1,Postini,web,"{'city': 'New York', 'latitude': None, 'longit..."
...,...,...,...
3782,Optemo,web,NaN
3783,Zoomission,web,NaN
3784,Eazeeloans,web,NaN
3785,DocASAP,web,NaN


In [15]:
full_address = result1[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)

In [16]:
full_address

,city,latitude,longitude
0,San Jose,37.321798,-121.979453
0,London,51.500152,-0.126236
1,Los Angeles,37.090240,-95.712891
1,London,NaN,NaN
1,New York,NaN,NaN
...,...,...,...
3782,NaN,NaN,NaN
3783,NaN,NaN,NaN
3784,NaN,NaN,NaN
3785,NaN,NaN,NaN


In [141]:
# We have the final structure of the dataframe with the correct columns and separations of values
join = pd.concat([result1, full_address], axis=1)

In [18]:
join

,name,category_code,offices,city,latitude,longitude
0,Wetpaint,web,"{'city': 'San Jose', 'latitude': 37.321798, 'l...",San Jose,37.321798,-121.979453
0,Wetpaint,web,"{'city': 'London', 'latitude': 51.500152, 'lon...",London,51.500152,-0.126236
1,Postini,web,"{'city': 'Los Angeles', 'latitude': 37.09024, ...",Los Angeles,37.090240,-95.712891
1,Postini,web,"{'city': 'London', 'latitude': None, 'longitud...",London,NaN,NaN
1,Postini,web,"{'city': 'New York', 'latitude': None, 'longit...",New York,NaN,NaN
...,...,...,...,...,...,...
3782,Optemo,web,NaN,NaN,NaN,NaN
3783,Zoomission,web,NaN,NaN,NaN,NaN
3784,Eazeeloans,web,NaN,NaN,NaN,NaN
3785,DocASAP,web,NaN,NaN,NaN,NaN


In [19]:
join = join.drop(columns = "offices")

In [20]:
join.loc[join.city == 'London', 'city'].count()


112

In [21]:
#I check which other cities are there
join.city.unique()

array(['San Jose', 'London', 'Los Angeles', 'New York', 'Chicago',
       'Toronto', 'Singapore', 'Zurich', 'Chiyoda-ku Tokyo',
       'San Fransisco', 'Moscow', 'Munich', 'Hong Kong', 'Paris',
       'San Francisco', 'Sydney', 'Montreal', 'Ljubljana', 'Stockholm',
       '', 'Madrid', 'Hamburg', 'Lod', 'Cologne', 'Celebration',
       'New York City', 'Beijing', 'Dublin', 'Waterford', 'Edinburgh',
       'Cambridge', 'Christchurch', 'Houston', 'Atlanta', 'Dallas',
       'Denver', 'Costa Mesa', 'Ft. Lauderdale', 'Seattle', 'Herndon',
       'Tokyo', 'Jersey City', 'Chao Yang, Beijing', 'Milton Keynes', nan],
      dtype=object)

In [142]:
#Because I only want London I drop all the other cities
Lnd_offices = join[(join.city != 'San Jose') & (join.city != 'Los Angeles') & (join.city != 'New York') & (join.city != 'Chicago') & (join.city != 'Toronto') & (join.city != 'Singapore') & (join.city != 'Zurich') & (join.city != 'Chiyoda-ku Tokyo') & (join.city != 'San Francisco') & (join.city != 'Moscow') & (join.city != 'Munich') & (join.city != 'Hong Kong')  & (join.city != 'Paris')  & (join.city != 'Sydney') & (join.city != 'Montreal')  & (join.city != 'Ljubljana') & (join.city != 'Stockholm')  & (join.city != '')  & (join.city != 'Madrid')  & (join.city != 'Hamburg') & (join.city != 'Lod') & (join.city != 'Cologne') & (join.city != 'Celebration') & (join.city != 'New York City') & (join.city != 'Beijing')  & (join.city != 'Dublin') & (join.city != 'Waterford') & (join.city != 'Edinburgh')  & (join.city != 'Cambridge') & (join.city != 'Christchurch') & (join.city != 'Houston') & (join.city != 'Atlanta') & (join.city != 'Dallas') & (join.city != 'Denver') & (join.city != 'Costa Mesa') & (join.city != 'Ft. Lauderdale') & (join.city != 'Seattle') & (join.city != 'Herndon') & (join.city != 'Tokyo') & (join.city != 'Jersey City') & (join.city != 'Chao Yang') & (join.city != 'Beijing') & (join.city != 'Milton Keynes')] 

In [23]:
#I drop the null values
Lnd_offices = Lnd_offices.dropna()
Lnd_offices

,name,category_code,city,latitude,longitude
0,Wetpaint,web,London,51.500152,-0.126236
2,Geni,web,London,37.090240,-95.712891
3,Fox Interactive Media,web,London,51.519204,-0.162610
4,StumbleUpon,web,London,37.090240,-95.712891
5,Gizmoz,web,London,51.523789,-0.087432
...,...,...,...,...,...
103,Tangler,web,London,51.511444,-0.135691
105,BillMonk,web,London,51.514735,-0.087424
106,TalkPlus,web,London,51.579373,-0.116242
107,MeeMix,web,London,51.521528,-0.103822


In [24]:
Lnd_offices.city.unique()

array(['London'], dtype=object)

I will choose an office and see if it could be an appropiate location for our new headquarters

In [25]:
print(Lnd_offices.loc[0])


name              Wetpaint
category_code          web
city                London
latitude         51.500152
longitude        -0.126236
Name: 0, dtype: object


In [26]:
#I use the location of the company named Wetpaint
office_lat = 51.500152
office_long = -0.126236

In [27]:
office = [("Santiago's Office","51.500152","-0.126236", "POINT (-0.126236 51.500152)", "Office Location", "Office")]
df_office = pd.DataFrame(office) 
df_office

,0,1,2,3,4,5
0,Santiago's Office,51.500152,-0.126236,POINT (-0.126236 51.500152),Office Location,Office


In [28]:
df_office.columns = ['name', 'latitude', 'longitude', 'geometry', 'criteria', 'category'] 
df_office

,name,latitude,longitude,geometry,criteria,category
0,Santiago's Office,51.500152,-0.126236,POINT (-0.126236 51.500152),Office Location,Office


I start the visualization of my office

In [153]:
map_1 = folium.Map(location = [office_lat,office_long], zoom_start=14)
map_1

In [154]:
icono = Icon(color = "darkblue",
             prefix = "fa",
             icon = "home",
             icon_color = "white",
             tooltip = "Possible location")

loc = {"location":[office_lat,office_long],
      "tooltip": "Possible location"}

marker = Marker(**loc, icon = icono)

marker.add_to(map_1)
map_1

In [32]:
city = "London"
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    try:
        return {
            "type":"Point",
            "coordinates":[float(data["longt"]),float(data["latt"])]}
    except:
        return data

donde = "London"
data = requests.get(f"https://geocode.xyz/{donde}?json=1").json()
print(data)

{'standard': {'addresst': {}, 'city': 'London', 'prov': 'UK', 'countryname': 'United Kingdom', 'postal': {}, 'confidence': '0.90'}, 'longt': '-0.11214', 'alt': {'loc': [{'longt': '-0.141590000000003', 'prov': 'UK', 'city': 'LONDON', 'postal': 'SW1E5RS', 'score': '1898', 'latt': '51.4974199999997'}, {'longt': '-0.14159', 'prov': 'UK', 'city': 'London', 'countryname': 'United Kingdom', 'postal': 'SW1E5RS', 'region': {}, 'latt': '51.49742'}]}, 'elevation': {}, 'latt': '51.51753'}


In [33]:
London = geocode(city)
London = {'type': 'Point', 'coordinates': [51.500152, -0.126236]}
print (London)

{'type': 'Point', 'coordinates': [51.500152, -0.126236]}


# Criteria 

In the previous step I obtained the location of what i want to be my future office. <br>
To make sure the location is suitable im going to look for a couple of criteria my employees need.

## Transportation

### Tube Stations

London is a busy city, with a lot of traffic. It is important for the account managers and other workers to be near the Tube station so they can easily move across the city.

In [157]:
# I use foursquare and I want the tube station to be very near so i look for ones within a 500m radius
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id='AJJNGXMPRIWHI41J1YCPPAAO3G3RUW4EWUQUF0KWCWE0WREX',
client_secret='QIGBJ0UEKJWY1AGHHPNJSNQPBIDT1HSKCFUNQEM1LJFHC5VW',
v='20180323',
ll='51.500152,-0.126236',
query='tube',
radius=500
)
resp = requests.get(url=url, params=params)
tube = json.loads(resp.text)


In [158]:
tube

{'meta': {'code': 200, 'requestId': '602528dacfd3f91592dd8e20'},
 'response': {'headerLocation': 'Westminster',
  'headerFullLocation': 'Westminster, London',
  'headerLocationGranularity': 'neighborhood',
  'query': 'tube',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 51.5046520045,
    'lng': -0.11902071992049519},
   'sw': {'lat': 51.4956519955, 'lng': -0.1334512800795048}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ac518cef964a520f6a520e3',
       'name': 'Elizabeth Tower (Big Ben) (Big Ben (Elizabeth Tower))',
       'contact': {},
       'location': {'address': 'Parliament Sq',
        'crossStreet': 'Bridge St',
        'lat': 51.50062,
        'lng': -0.124578,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.50062,
         

In [37]:
name = tube["response"]["groups"][0]["items"][0]["venue"]["name"] 
latitud = tube["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
longitud = tube["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]


print(name, ":" , latitud,longitud)

Elizabeth Tower (Big Ben) (Big Ben (Elizabeth Tower)) : 51.50062 -0.124578


In [38]:
tube_distance=tube["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
print(f"{tube_distance} m")

126 m


The nearest Tube Station to our office is at Elizabeth Tower, only 126m away. <br>
Perfect for our workers.

In [39]:
tube = [[i['venue']['name'],i['venue']['location']['lat'],i['venue']['location']['lng']] for i in tube['response']['groups'][0]['items']]

In [40]:
location_tube=[]
for i in range(len(tube)):
        loc = {"tube":tube[i]}
        location_tube.append(loc)

I create a dataframe with the Tube Stations

In [41]:
tube_df=pd.DataFrame(location_tube)
tube_df.head()

,tube
0,[Elizabeth Tower (Big Ben) (Big Ben (Elizabeth...
1,"[Westminster London Underground Station, 51.50..."
2,"[The Feathers, 51.499389882878205, -0.13308796..."
3,"[St. Stephens Tavern, 51.50104512723709, -0.12..."


In [42]:
len(tube_df)

4

There are 4 tube stations within a 500m radius

In [43]:
tube_df[["name","latitude","longitude"]] = pd.DataFrame(tube_df.tube.tolist(), index= tube_df.index)
tube_df = tube_df.drop(['tube'], axis=1)
tube_df['category'] = 'tube'
tube_df


,name,latitude,longitude,category
0,Elizabeth Tower (Big Ben) (Big Ben (Elizabeth ...,51.500620,-0.124578,tube
1,Westminster London Underground Station,51.500922,-0.124802,tube
2,The Feathers,51.499390,-0.133088,tube
3,St. Stephens Tavern,51.501045,-0.125511,tube


In [159]:
# I convert my data frame into a Geodataframe
gdf_tube = gpd.GeoDataFrame(tube_df, geometry = gpd.points_from_xy(tube_df.longitude,tube_df.latitude))
gdf_tube.head()

,name,latitude,longitude,category,geometry
0,Elizabeth Tower (Big Ben) (Big Ben (Elizabeth ...,51.500620,-0.124578,tube,POINT (-0.12458 51.50062)
1,Westminster London Underground Station,51.500922,-0.124802,tube,POINT (-0.12480 51.50092)
2,The Feathers,51.499390,-0.133088,tube,POINT (-0.13309 51.49939)
3,St. Stephens Tavern,51.501045,-0.125511,tube,POINT (-0.12551 51.50105)


In [46]:
#I create a simple map to have an idea of where the Tube Stations are.
Map(Layer(gdf_tube, popup_hover = [popup_element("name", "Tubes")]))

### Airport

I will look for an airport nearby, to for the managers to be able to travel if there are any meetings abroad.

In [205]:
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id='AJJNGXMPRIWHI41J1YCPPAAO3G3RUW4EWUQUF0KWCWE0WREX',
client_secret='QIGBJ0UEKJWY1AGHHPNJSNQPBIDT1HSKCFUNQEM1LJFHC5VW',
v='20180323',
ll='51.500152,-0.126236',
query='airport',
)
resp = requests.get(url=url, params=params)
airport = json.loads(resp.text)

In [48]:
airport

{'meta': {'code': 200, 'requestId': '60240ad00ac61561dc9b8957'},
 'response': {'suggestedRadius': 882,
  'headerLocation': 'Westminster',
  'headerFullLocation': 'Westminster, London',
  'headerLocationGranularity': 'neighborhood',
  'query': 'airport',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 51.51526045383694,
    'lng': -0.12274752694244384},
   'sw': {'lat': 51.4672244694244, 'lng': -0.4785199342086792}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad7a112f964a520050d21e3',
       'name': 'London Heathrow Airport (LHR) (London Heathrow Airport)',
       'contact': {},
       'location': {'address': 'Nelson Rd',
        'lat': 51.469407923261336,
        'lng': -0.46234846115112305,
        'labeledLatLngs': [{'label': 'display',
          'lat': 

In [49]:
name = airport["response"]["groups"][0]["items"][0]["venue"]["name"] 
latitud = airport["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
longitud = airport["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]


print(name, ":" , latitud,longitud)

London Heathrow Airport (LHR) (London Heathrow Airport) : 51.469407923261336 -0.46234846115112305


In [50]:
airport_distance=airport["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
print(f"{airport_distance} m")

23549 m


The nearest airport to our office is 23.5Km away.

In [51]:
airport = [[i['venue']['name'],i['venue']['location']['lat'],i['venue']['location']['lng']] for i in airport['response']['groups'][0]['items']]

In [52]:
location_airport=[]
for i in range(len(airport)):
        loc = {"airport":airport[i]}
        location_airport.append(loc)

In [53]:
airport_df = pd.DataFrame(location_airport)
airport_df

,airport
0,[London Heathrow Airport (LHR) (London Heathro...
1,"[No.1 Traveller, 51.513077, -0.138919]"
2,"[National Audit Office, 51.49173283611784, -0...."
3,"[HM Passport Office, 51.49349038356057, -0.144..."


In [54]:
airport_df[["name","latitude","longitude"]] = pd.DataFrame(airport_df.airport.tolist(), index= airport_df.index)
airport_df = airport_df.drop(['airport'], axis=1)
airport_df['category'] = 'airport'
airport_df.head()


,name,latitude,longitude,category
0,London Heathrow Airport (LHR) (London Heathrow...,51.469408,-0.462348,airport
1,No.1 Traveller,51.513077,-0.138919,airport
2,National Audit Office,51.491733,-0.148453,airport
3,HM Passport Office,51.493490,-0.144184,airport


London Heathrow is the neares airport from our office at about 23.5km. <br>
Due to the good accesibility to tube stations from our office it shouldn't be hard to get there.


Overall, our office is located in a position with close access to the subway system, and therefore it has good connection with the rest of the city and the airport. <br>
Our office location passes our first criteria of **TRANSPORTATION**

# Vets

Dogs are humans best friend (and maybe cats or other pets for some) so we want to make sure there is a veterinary nearby so the workers can keep their pets healthy

In [176]:
# I use foursquare and want to see how many vets are within a 3000m radius of our office.
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id='AJJNGXMPRIWHI41J1YCPPAAO3G3RUW4EWUQUF0KWCWE0WREX',
client_secret='QIGBJ0UEKJWY1AGHHPNJSNQPBIDT1HSKCFUNQEM1LJFHC5VW',
v='20180323',
ll='51.500152,-0.126236',
query='veterinary',
radius=3000
)
resp = requests.get(url=url, params=params)
vet= json.loads(resp.text)

In [177]:
vet

{'meta': {'code': 200, 'requestId': '60254d9b77c0b13d3460e6af'},
 'response': {'headerLocation': 'London',
  'headerFullLocation': 'London',
  'headerLocationGranularity': 'city',
  'query': 'veterinary',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 51.52715202700003,
    'lng': -0.08294431952297118},
   'sw': {'lat': 51.47315197299997, 'lng': -0.1695276804770288}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb1bd37f964a5200da13ce3',
       'name': 'The Blue Cross Animal Hospital',
       'contact': {},
       'location': {'address': 'Hugh St.',
        'crossStreet': 'Victoria St.',
        'lat': 51.49304190285976,
        'lng': -0.14464702272395094,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.49304190285976,
          'lng': 

In [178]:
name = vet["response"]["groups"][0]["items"][0]["venue"]["name"] 
latitud = vet["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
longitud = vet["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]


print(name, ":" , latitud,longitud)

The Blue Cross Animal Hospital : 51.49304190285976 -0.14464702272395094


In [179]:
vet_distance=vet["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
print(f"{vet_distance} m")

1501 m


The nearest vet is 1.5Km away.

In [180]:
vet = [[i['venue']['name'],i['venue']['location']['lat'],i['venue']['location']['lng']] for i in vet['response']['groups'][0]['items']]

In [181]:
location_vet=[]
for i in range(len(vet)):
        loc = {"vet":vet[i]}
        location_vet.append(loc)

In [182]:
location_vet

[{'vet': ['The Blue Cross Animal Hospital',
   51.49304190285976,
   -0.14464702272395094]},
 {'vet': ['Goddard Veterinary Group, Kennington', 51.487169, -0.1113]},
 {'vet': ['Goddards Veterinary Practice', 51.487159, -0.111202]},
 {'vet': ['Elizabeth Street Veterinary Clinic', 51.49369776248932, -0.150854]}]

In [183]:
vet_df = pd.DataFrame(location_vet)
vet_df

,vet
0,"[The Blue Cross Animal Hospital, 51.4930419028..."
1,"[Goddard Veterinary Group, Kennington, 51.4871..."
2,"[Goddards Veterinary Practice, 51.487159, -0.1..."
3,"[Elizabeth Street Veterinary Clinic, 51.493697..."


There are 4 veterinary's that are within 3000m of our office.

In [184]:
vet_df[["name","latitude","longitude"]] = pd.DataFrame(vet_df.vet.tolist(), index= vet_df.index)
vet_df = vet_df.drop(['vet'], axis=1)
vet_df['category'] = 'Veterinary'
vet_df.head()

,name,latitude,longitude,category
0,The Blue Cross Animal Hospital,51.493042,-0.144647,Veterinary
1,"Goddard Veterinary Group, Kennington",51.487169,-0.111300,Veterinary
2,Goddards Veterinary Practice,51.487159,-0.111202,Veterinary
3,Elizabeth Street Veterinary Clinic,51.493698,-0.150854,Veterinary


In [185]:
gdf_vet = gpd.GeoDataFrame(vet_df, geometry = gpd.points_from_xy(vet_df.longitude,vet_df.latitude))
gdf_vet.head()

,name,latitude,longitude,category,geometry
0,The Blue Cross Animal Hospital,51.493042,-0.144647,Veterinary,POINT (-0.14465 51.49304)
1,"Goddard Veterinary Group, Kennington",51.487169,-0.111300,Veterinary,POINT (-0.11130 51.48717)
2,Goddards Veterinary Practice,51.487159,-0.111202,Veterinary,POINT (-0.11120 51.48716)
3,Elizabeth Street Veterinary Clinic,51.493698,-0.150854,Veterinary,POINT (-0.15085 51.49370)


In [186]:
Map(Layer(gdf_vet, popup_hover = [popup_element("name", "Vets")]))

Londoner's a known for their love of their pets, and as such having the Blue Cross Animal Hospital only 1501 meters away from our office is a great asset. Also, there are a further 3 veterinary practices within 3000 meters

Our office location passes our second criteria of having a **VETERINARY** near so the workers can take their pets and keep them healthy.

# Starbucks

Everybody loves a good coffee, so I'm going to make sure there is a Starbucks nearby.

In [200]:
# I look for a Starubcks within a 1000m radius.
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id='AJJNGXMPRIWHI41J1YCPPAAO3G3RUW4EWUQUF0KWCWE0WREX',
client_secret='QIGBJ0UEKJWY1AGHHPNJSNQPBIDT1HSKCFUNQEM1LJFHC5VW',
v='20180323',
ll='51.500152,-0.126236',
query='Starbucks',
radius=1000
)
resp = requests.get(url=url, params=params)
starbucks= json.loads(resp.text)

In [201]:
starbucks

{'meta': {'code': 200, 'requestId': '602550c08e5f3a3826cf2f8e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Westminster',
  'headerFullLocation': 'Westminster, London',
  'headerLocationGranularity': 'neighborhood',
  'query': 'starbucks',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 51.50915200900001,
    'lng': -0.11180543984099038},
   'sw': {'lat': 51.49115199099999, 'lng': -0.14066656015900958}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bc0539e920eb71357ca182c',
       'name': 'Starbucks',
       'contact': {},
       'location': {'address': 'Belvedere Road, 3C and 3D, East Block County Hall',
        'crossStreet': 'Belvedere Rd',
        'lat': 51.50

In [202]:
name = starbucks["response"]["groups"][0]["items"][0]["venue"]["name"] 
latitud = starbucks["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
longitud = starbucks["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]


print(name, ":" , latitud,longitud)

Starbucks : 51.501962 -0.117661


In [203]:
starbucks_distance=starbucks["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
print(f"{starbucks_distance} m")

627 m


Th nearest starbucks is 627 meters away

In [204]:
starbucks = [[i['venue']['name'],i['venue']['location']['lat'],i['venue']['location']['lng']] for i in starbucks['response']['groups'][0]['items']]

In [84]:
location_starbucks=[]
for i in range(len(starbucks)):
        loc = {"starbucks":starbucks[i]}
        location_starbucks.append(loc)
location_starbucks

[{'starbucks': ['Starbucks', 51.501962, -0.117661]},
 {'starbucks': ['Starbucks', 51.499001, -0.135474]},
 {'starbucks': ['Starbucks', 51.498196, -0.132676]},
 {'starbucks': ['Starbucks', 51.50745, -0.122863]},
 {'starbucks': ['Starbucks', 51.499107, -0.133562]},
 {'starbucks': ['Starbucks', 51.503119592683895, -0.11298537254333496]}]

In [85]:
starbucks_df = pd.DataFrame(location_starbucks)
starbucks_df

,starbucks
0,"[Starbucks, 51.501962, -0.117661]"
1,"[Starbucks, 51.499001, -0.135474]"
2,"[Starbucks, 51.498196, -0.132676]"
3,"[Starbucks, 51.50745, -0.122863]"
4,"[Starbucks, 51.499107, -0.133562]"
5,"[Starbucks, 51.503119592683895, -0.11298537254..."


In [86]:
starbucks_df[["name","latitude","longitude"]] = pd.DataFrame(starbucks_df.starbucks.tolist(), index= starbucks_df.index)
starbucks_df = starbucks_df.drop(['starbucks'], axis=1)
starbucks_df['category'] = 'starbucks'
starbucks_df

,name,latitude,longitude,category
0,Starbucks,51.501962,-0.117661,starbucks
1,Starbucks,51.499001,-0.135474,starbucks
2,Starbucks,51.498196,-0.132676,starbucks
3,Starbucks,51.507450,-0.122863,starbucks
4,Starbucks,51.499107,-0.133562,starbucks
5,Starbucks,51.503120,-0.112985,starbucks


In [87]:
len(starbucks_df)

6

There are 6 Starbucks locations within 1km from the possible office.

In [88]:
gdf_starbucks = gpd.GeoDataFrame(starbucks_df, geometry = gpd.points_from_xy(starbucks_df.longitude,starbucks_df.latitude))
gdf_starbucks

,name,latitude,longitude,category,geometry
0,Starbucks,51.501962,-0.117661,starbucks,POINT (-0.11766 51.50196)
1,Starbucks,51.499001,-0.135474,starbucks,POINT (-0.13547 51.49900)
2,Starbucks,51.498196,-0.132676,starbucks,POINT (-0.13268 51.49820)
3,Starbucks,51.507450,-0.122863,starbucks,POINT (-0.12286 51.50745)
4,Starbucks,51.499107,-0.133562,starbucks,POINT (-0.13356 51.49911)
5,Starbucks,51.503120,-0.112985,starbucks,POINT (-0.11299 51.50312)


In [90]:
Map(Layer(gdf_starbucks, popup_hover = [popup_element("name", "Starbucks")]))

I have looked at the Starbucks locations within 1000 meters of our office and found there are 6, the nearest one being 627 meters away.

This result meets our criteria of having a **STARBUCKS** near, as a coffee should be quick it is a good thing that they are close by. <br>
We also have plenty of options, so if one is full we can simply go to the next to avoid queues.

# Basketball Stadium

The maintenance employee is a big fan of Basketball, so I will look for a court nearby so he can go and enjoy playing.

In [207]:
#I look for courts within a 2000m radius.

url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id='AJJNGXMPRIWHI41J1YCPPAAO3G3RUW4EWUQUF0KWCWE0WREX',
client_secret='QIGBJ0UEKJWY1AGHHPNJSNQPBIDT1HSKCFUNQEM1LJFHC5VW',
v='20180323',
ll='51.500152,-0.126236',
query='Basketball court',
radius=2000
)
resp = requests.get(url=url, params=params)
basketball= json.loads(resp.text)

In [193]:
basketball

{'meta': {'code': 200, 'requestId': '602550902776b86a58f3c865'},
 'response': {'warning': {'text': 'There aren\'t a lot of results for "Basketball court." Try something more general, reset your filters, or expand the search area.'},
  'headerLocation': 'Westminster',
  'headerFullLocation': 'Westminster, London',
  'headerLocationGranularity': 'neighborhood',
  'query': 'basketball court',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 51.518152018000016,
    'lng': -0.09737487968198077},
   'sw': {'lat': 51.48215198199998, 'lng': -0.1550971203180192}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ba5e970f964a520ef2839e3',
       'name': 'Vauxhall Park',
       'contact': {},
       'location': {'address': 'Fentiman Rd',
        'lat': 51.48273255637877,
  

In [194]:
name = basketball["response"]["groups"][0]["items"][0]["venue"]["name"] 
latitud = basketball["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
longitud = basketball["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]


print(name, ":" , latitud,longitud)

Vauxhall Park : 51.48273255637877 -0.12217998504638672


In [196]:
basketball_distance=basketball["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
print(f"{basketball_distance} m")

1959 m


The nearest Basketball court is 1959 meters from the office.

In [197]:
basketball = [[i['venue']['name'],i['venue']['location']['lat'],i['venue']['location']['lng']] for i in basketball['response']['groups'][0]['items']]

In [174]:
location_basketball=[]
for i in range(len(basketball)):
        loc = {"basketball":basketball[i]}
        location_basketball.append(loc)
location_basketball

[{'basketball': ['Vauxhall Park', 51.48273255637877, -0.12217998504638672]},
 {'basketball': ['The Regal', 51.491417168677145, -0.11480698376044421]},
 {'basketball': ['Geraldine Mary Harmsworth Sports Facility',
   51.49618725041645,
   -0.10674841046899525]}]

In [97]:
basketball_df = pd.DataFrame(location_basketball)
basketball_df

,basketball
0,"[Vauxhall Park, 51.48273255637877, -0.12217998..."
1,"[The Regal, 51.491417168677145, -0.11480698376..."
2,"[Geraldine Mary Harmsworth Sports Facility, 51..."


There are 3 Basketball courts within a 2km radius

In [98]:
basketball_df[["name","latitude","longitude"]] = pd.DataFrame(basketball_df.basketball.tolist(), index= basketball_df.index)
basketball_df = basketball_df.drop(['basketball'], axis=1)
basketball_df['category'] = 'basketball'
basketball_df

,name,latitude,longitude,category
0,Vauxhall Park,51.482733,-0.122180,basketball
1,The Regal,51.491417,-0.114807,basketball
2,Geraldine Mary Harmsworth Sports Facility,51.496187,-0.106748,basketball


In [99]:
gdf_basketball = gpd.GeoDataFrame(basketball_df, geometry = gpd.points_from_xy(basketball_df.longitude,basketball_df.latitude))
gdf_basketball

,name,latitude,longitude,category,geometry
0,Vauxhall Park,51.482733,-0.122180,basketball,POINT (-0.12218 51.48273)
1,The Regal,51.491417,-0.114807,basketball,POINT (-0.11481 51.49142)
2,Geraldine Mary Harmsworth Sports Facility,51.496187,-0.106748,basketball,POINT (-0.10675 51.49619)


In [101]:
Map(Layer(gdf_basketball, popup_hover = [popup_element("name", "Basketball Court")]))

The maintenance guy will be happy knowing there are 3 possible courts nearby where he could play Basketball. The nearest one is 1950 meters away, so that means the other 2 are not much further. <br>
This means that our criteria of a **BASKETBALL COURT** is met.

# Vegan restaurant

Lastly, we want to make sure that there is a vegan restaurant nearby. <br>
The CEO is vegan, so it is essential.

In [161]:
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id='AJJNGXMPRIWHI41J1YCPPAAO3G3RUW4EWUQUF0KWCWE0WREX',
client_secret='QIGBJ0UEKJWY1AGHHPNJSNQPBIDT1HSKCFUNQEM1LJFHC5VW',
v='20180323',
ll='51.500152,-0.126236',
query='vegan restaurant',
radius=1500
)
resp = requests.get(url=url, params=params)
vegan= json.loads(resp.text)

In [162]:
vegan

{'meta': {'code': 200, 'requestId': '602542a39854e8099c0bb981'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Westminster',
  'headerFullLocation': 'Westminster, London',
  'headerLocationGranularity': 'neighborhood',
  'query': 'vegan restaurant',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 51.513652013500014,
    'lng': -0.10459015976148558},
   'sw': {'lat': 51.486651986499986, 'lng': -0.1478818402385144}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b326cd9f964a520630b25e3',
       'name': 'Sagar',
       'contact': {},
       'location': {'address': '31 Catherine St, Hammersmith',
        'lat': 51.5126203,
        'lng': -0.1207112,
        'labeledLatLn

In [163]:
name = vegan["response"]["groups"][0]["items"][0]["venue"]["name"] 
latitud = vegan["response"]["groups"][0]["items"][0]["venue"]["location"]["lat"]
longitud = vegan["response"]["groups"][0]["items"][0]["venue"]["location"]["lng"]


print(name, ":" , latitud,longitud)

Sagar : 51.5126203 -0.1207112


In [164]:
restaurant_distance=vegan["response"]["groups"][0]["items"][0]["venue"]["location"]["distance"]
print(f"{restaurant_distance} m")

1439 m


The nearest vegan restaurant is 1.4km away.

In [165]:
vegan= [[i['venue']['name'],i['venue']['location']['lat'],i['venue']['location']['lng']] for i in vegan['response']['groups'][0]['items']]

In [166]:
location_restaurant=[]
for i in range(len(vegan)):
        loc = {"vegan":vegan[i]}
        location_restaurant.append(loc)
location_restaurant[:2]

[{'vegan': ['Sagar', 51.5126203, -0.1207112]},
 {'vegan': ['Woodlands Restaurant', 51.50960054761273, -0.13178168153152411]}]

In [167]:
restaurant_df = pd.DataFrame(location_restaurant)

In [168]:
restaurant_df[["name","latitude","longitude"]] = pd.DataFrame(restaurant_df.vegan.tolist(), index= restaurant_df.index)
restaurant_df = restaurant_df.drop(['vegan'], axis=1)
restaurant_df['category'] = 'Vegan Restaurant'
restaurant_df

,name,latitude,longitude,category
0,Sagar,51.512620,-0.120711,Vegan Restaurant
1,Woodlands Restaurant,51.509601,-0.131782,Vegan Restaurant
2,Maoz,51.512861,-0.132200,Vegan Restaurant
3,Ragged Canteen,51.491893,-0.118984,Vegan Restaurant
4,Abokado,51.496904,-0.133805,Vegan Restaurant
5,Crust Bros Pizza,51.502768,-0.110457,Vegan Restaurant
6,Vantra Loungevity,51.510611,-0.131857,Vegan Restaurant
7,Soho Vegan Market,51.512285,-0.133766,Vegan Restaurant
8,Fitology Kitchen,51.507468,-0.110085,Vegan Restaurant
9,Misato,51.510896,-0.131654,Vegan Restaurant


There are 10 vegan restaurants within a 1500 meter radius from the office.

In [169]:
gdf_restaurant = gpd.GeoDataFrame(restaurant_df, geometry = gpd.points_from_xy(restaurant_df.longitude,restaurant_df.latitude))
gdf_restaurant

,name,latitude,longitude,category,geometry
0,Sagar,51.512620,-0.120711,Vegan Restaurant,POINT (-0.12071 51.51262)
1,Woodlands Restaurant,51.509601,-0.131782,Vegan Restaurant,POINT (-0.13178 51.50960)
2,Maoz,51.512861,-0.132200,Vegan Restaurant,POINT (-0.13220 51.51286)
3,Ragged Canteen,51.491893,-0.118984,Vegan Restaurant,POINT (-0.11898 51.49189)
4,Abokado,51.496904,-0.133805,Vegan Restaurant,POINT (-0.13381 51.49690)
5,Crust Bros Pizza,51.502768,-0.110457,Vegan Restaurant,POINT (-0.11046 51.50277)
6,Vantra Loungevity,51.510611,-0.131857,Vegan Restaurant,POINT (-0.13186 51.51061)
7,Soho Vegan Market,51.512285,-0.133766,Vegan Restaurant,POINT (-0.13377 51.51228)
8,Fitology Kitchen,51.507468,-0.110085,Vegan Restaurant,POINT (-0.11009 51.50747)
9,Misato,51.510896,-0.131654,Vegan Restaurant,POINT (-0.13165 51.51090)


In [171]:
Map(Layer(gdf_restaurant, popup_hover = [popup_element("name", "Vegan Restaurant")]))

The nearest vegan restaurant to our office is Sagar, located 1439 meters away from our office. In addition to this, there are 9 other vegan restaurants near, so the CEO and any other vegan employees have plenty of choice of where to go to it. <br>
This means that our criteria of a **VEGAN RESTAURANT** is met again.

# MAP

We have looked at 5 different criteria, we had to be near a Tube station, Starbucks, Veterinary, Basketball Court, and a Vegan Restaurant, and all of that criteria has been met. Therefore, the possible location that we had seen above is suitable to create our headquarters.

In [208]:
# I create a dataframe with all of the locations of the criteria that was found above.
concat_df = pd.concat([df_office,gdf_tube, gdf_starbucks, gdf_vet, gdf_basketball, gdf_restaurant])
concat_df.head()

,name,latitude,longitude,geometry,criteria,category
0,Santiago's Office,51.500152,-0.126236,POINT (-0.126236 51.500152),Office Location,Office
0,Elizabeth Tower (Big Ben) (Big Ben (Elizabeth ...,51.500620,-0.124578,POINT (-0.124578 51.50062),NaN,tube
1,Westminster London Underground Station,51.500922,-0.124802,POINT (-0.1248020513432981 51.50092199879671),NaN,tube
2,The Feathers,51.499390,-0.133088,POINT (-0.13308796236854 51.49938988287821),NaN,tube
3,St. Stephens Tavern,51.501045,-0.125511,POINT (-0.1255105271062834 51.50104512723709),NaN,tube


In [114]:
concat_df.category.unique()

array(['Office', 'tube', 'starbucks', 'vet', 'basketball', 'restaurant'],
      dtype=object)

In [155]:
# Set the points in the map
for i,row in concat_df.iterrows():
    name = {"location" : [row["latitude"],row["longitude"]],
            "tooltip" : row["name"]}
    
    if row["category"] == "tube":
        icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "subway",
                    icon_color = "white")
    elif row["category"] == "starbucks":
        icon = Icon(color = "white",
                    prefix = "fa",
                    icon = "coffee",
                    icon_color = "green")   
    elif row["category"] == "vet":
        icon = Icon(color = "black",
                    prefix = "fa",
                    icon = "paw",
                    icon_color = "white")
    elif row["category"] == "basketball":
        icon = Icon(color = "orange",
                    prefix = "fa",
                    icon = "futbol-o",
                    icon_color = "white")
    elif row["category"] == "restaurant":
        icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "leaf",
                    icon_color = "orange")

        
    
        Marker(**name,icon = icon).add_to(map_1)

In [151]:
#Final Map with all the points and locations.
map_1

This map shows our office as the blue spot with a *home* icon, the Tube stations are shown with light blue and a *subway* icon, the vet has the *paw* icon, Starbucks the *coffee mug* icon and the vegan restaurant has the orange *leaf* icon.